# 単純ベイズ

sklearn.naive_bayesを用いて、スパム分類器を簡単に作りましょう。大部分は、訓練用のデータをpandasのDataFrameに入れるコードです。

In [1]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('c:/DataScience/DataScience/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('c:/DataScience/DataScience/emails/ham', 'ham'))


DataFrameを見てみましょう。

In [2]:
data.head()

,class,message
c:/DataScience/DataScience/emails/spam\00001.7848dde101aa985090474a91ec93fcf0,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
c:/DataScience/DataScience/emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
c:/DataScience/DataScience/emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
c:/DataScience/DataScience/emails/spam\00004.eac8de8d759b7e74154f142194282724,spam,##############################################...
c:/DataScience/DataScience/emails/spam\00005.57696a39d7d84318ce497886896bf90d,spam,I thought you might like these:\n\n1) Slim Dow...


CountVectorizerを用いて、各メッセージを各単語数を表すベクトルに変換します。これを各メッセージに対応するclassのデータと共にMultinomialNB分類器に入れて、fit関数により訓練を行います。 これにより、訓練済みのスパムフィルターを得ることができます。 

In [3]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

それでは、適当なメッセージを分類してみましょう！

In [4]:
examples = ['Earn 10000$!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], 
      dtype='|S4')

## アクティビティ

今回はデータが少ないため、スパム分類器の精度はあまりよくありませんが、examplesに他のメッセージを追加してみて、期待するような結果が得られるかどうか試してみましょう。

もしやる気があるのであれば、訓練とテストをこのスパム分類器で試してみましょう。スパム分類器の精度が数値化されるでしょう。